In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

%matplotlib widget

# 1) Data: 2D moon + injected outliers
rng = np.random.default_rng(42)
X, _ = make_moons(n_samples=600, noise=0.08, random_state=0)
n_out = 15
outliers = rng.uniform(low=[-2.0, -1.5], high=[2.5, 2.0], size=(n_out, 2))
X_all = np.vstack([X, outliers])

fig,ax = plt.subplots()
ax.scatter(X_all[:,0], X_all[:,1], lw = 1, edgecolors='k', s = 20)
ax.set_xlabel('x1')
ax.set_ylabel('x2')

In [ ]:
# 2) Fit IsolationForest
iso = IsolationForest(
    n_estimators=200, max_samples=256, contamination=0.02,
    random_state=42, n_jobs=-1
).fit(X_all)

# Anomaly score: higher = more normal in scikit (-score_ if you prefer)
score = iso.decision_function(X_all)  # ~0 normal, negative = outlier
pred = iso.predict(X_all)             # 1 = inlier, -1 = outlier

# 3) (Optional) quick comparator (unsupervised too)
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.02)
pred_lof = lof.fit_predict(X_all)  # 1 / -1

# 4) Plot decision function as a contour
xx, yy = np.meshgrid(np.linspace(-2.2, 3.0, 400), np.linspace(-1.8, 2.2, 400))
grid = np.c_[xx.ravel(), yy.ravel()]
zz = iso.decision_function(grid).reshape(xx.shape)

plt.figure(figsize=(7,5))
# Heatmap of normality
cs = plt.contourf(xx, yy, zz, levels=30, alpha=0.9)
plt.colorbar(cs, label="IsolationForest decision_function (↑ more normal)")

# Inliers vs outliers (according to IF)
inliers = X_all[pred == 1]
outs    = X_all[pred == -1]
plt.scatter(inliers[:,0], inliers[:,1], s=18, edgecolor='k', linewidths=0.2, label='inliers')
plt.scatter(outs[:,0],    outs[:,1],    s=60, marker='x', linewidths=2, label='IF outliers')

# Also show true injected outliers for drama
plt.scatter(outliers[:,0], outliers[:,1], s=120, facecolors='none', edgecolors='red', linewidths=2, label='injected')

plt.legend(loc='lower left')
plt.title("Anomaly detection with IsolationForest")
plt.xlabel("x1"); plt.ylabel("x2")
plt.tight_layout(); plt.show()

# 5) Try new points live
X_new = np.array([[0.5, 1.0], [2.0, 1.8], [-1.5, -1.2]])  # edit during the talk
print("New points →", X_new.tolist())
print("IF prediction (1=inlier, -1=outlier):", iso.predict(X_new).tolist())
print("IF decision_function:", np.round(iso.decision_function(X_new), 3).tolist())

# 6) Quick note on LOF agreement (optional)
agree = (pred_lof == pred)
print(f"LOF agrees with IF on {agree.mean()*100:.1f}% of points.")